In [ ]:
import openai
import os
import json
from datasets import load_dataset
openai.api_key = ''
dataset = load_dataset("openai_humaneval", split="test")

In [ ]:
def get_completion(prompt, model="gpt-4"):
    messages = [
        {"role": "system",
         "content": "You are a helpful code generation model \
         that can generate source code for a Python function \
         given its header and comments describing its purpose."},
        {"role": "user", "content": prompt},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message['content']

In [ ]:
def nl_to_code():
    with open ('he_sims.jsonl') as f:
        json_list = list(f)
    test_dict = []
    for json_str in json_list:
        result = json.loads(json_str)
        test_dict.append(result)
    
    write_file = open("humaneval_in_context.txt", "w")
    group_size = 4
    groups = zip(*(iter(dataset),) * group_size)
    for test, group in zip(test_dict, groups):        
        prompt = f"""
        Given 3 pairs of Python code examples and answers, \
        generate Python source code for 4 functions' bodies, \
        given Python code (surrounded by ???) containing imports, \
        the function's header, and comments describing the function's \
        purpose along with sample test cases & expected results.\
        Output only the Python function body (no header, comments, or imports) \ 
        on exactly one line. \
        Text 1: {test['nl'][0]} \
        Code 1: {test['code'][0]} \
        Text 2: {test['nl'][1]} \
        Code 2: {test['code'][1]} \
        Text 3: {test['nl'][2]} \
        Code 3: {test['code'][2]} \
        Code 1: ???{group[0]['prompt']}??? \
        Code 2: ???{group[1]['prompt']}??? \
        Code 3: ???{group[2]['prompt']}??? \
        Code 4: ???{group[3]['prompt']}??? \
        Answer 1: \
        Answer 2: \
        Answer 3: \
        Answer 4: \
        """
        responses = get_completion(prompt).splitlines()
        for response in responses:
            if len(response) != 0:
                write_file.write(response + '\n')
            

In [ ]:
nl_to_code()